## Coordinate Frames

Listed in order from the most user facing, to the back end internal raw data.

| Frame | Units         | Origin   | Y dir | Description |
| ----- | ------------- | -------- | ----- | ----------- |
| qwn   | window pixels | top left | down  | Qt window coordinates |
| ndc   | half windows  | center   | up    | OpenGL normalized device coordinates |
| nic   | half images   | center   | up    | vimage normalized reoriented isotropic image coordinates |
| omp   | image pixels  | top left | down  | rectified image pixels |
| obq   | radians       | center   | up    | "oblique" image sphere rotated to current view |
| ont   | radians       | center   | up    | image sphere rectified to canonical orientation using EXIF/XMP metadata |
| eqr   | radians       | center   | up    | heading, pitch (equirectangular projection) |
| txc   | images        | top left | down  | OpenGL texture coordinates |

## Transform parameters

1. Zoom: windows per image
2. Direction: heading, pitch degrees (360 only)
3. Image center: image pixel x, y (rectangular only)
4. Image width, height: image pixels (respecting EXIF orientation)
5. Window width, height: screen pixels

### Situations where transformation between coordinate frames is needed:

 1. When single dragging images to pan. Here we need the jacobian matrix relating qwn coordinates to view_model parameters. $viw_J_qwn$
     * image center (but not zoom), in the case of rectangular images
     * view pitch and heading angles (but not zoom nor roll) in the case of 360 images
 2. When rendering images in a GLSL shader. $txc_X_ndc$
 3. When hovering with the mouse $omp_X_qwn$ for rectangular and $viw_X_qwn$ for 360 images.

The only frames we need to compute are:
  * tex, in the rectangular shader tex_X_ndc
  * omp, on hover omp_X_qwn
  * derivative of image center (in either ont, raw, tex, img, or omp), on drag omp_J_qwn
    * ypr for 360s ypr_J_qwn
    * omp for rectangular omp_J_qwn
  
Rectangular view state:
  * image center (should be omp, TODO:)
  * zoom (windows per image)
360 view state:
  * pitch, yaw (maybe as equirect image center in degrees)
  * zoom (windows per image at center?)
  
What are normalized image coordinates (nic) ?
  * origin (0,0) is at image center
  * Y increases upward
  * EXIF reorientation is applied
  * coordinates range from -1 (left, bottom) to +1 (right, top)
  * scaled such that underlying full image exactly fits in the window. The edge touching dimension ranges from -1 to +1
  
Vertex shader converts from ndc to nic
  * apply zoom, window aspect, image size, image center
Fragment shader computes from nic to txc, and samples
  * apply pixel filter
  * 360 images:
    * nic to geocentric unit sphere
    * rotate by yaw pitch roll
    * project to projection...


In [11]:
from sympy import *
init_printing()

The transform required for hover coordinates is composed of sub-transforms:

$^{omp}p \ = \ ^{omp}X^{qwn} \ \cdot \ ^{qwn}p $

$^{omp}X^{qwn} \ = \ ^{omp}X^{nic} \ \cdot \ ^{nic}X^{ndc} \ \cdot \ ^{ndc}X^{qwn}$


In [12]:
# printed symbols of transforms
omp_X_qwn_s = symbols("^{omp}X^{qwn}")
ndc_X_qwn_s = symbols("^{ndc}X^{qwn}")
nic_X_ndc_s = symbols("^{nic}X^{ndc}")
omp_X_nic_s = symbols("^{omp}X^{nic}")

In [13]:
# ndc_X_qwn: Transform to OpenGL normalized device coordinates from Qt window pixel coordinates

# window dimensions
w_win, h_win = symbols("w_win, h_win")

ndc_X_qwn_m = Matrix([
    [2/w_win, 0, -1],
    [0, -2/h_win, 1],
    [0, 0, 1],
])

# confidence checks

# Upper left corner
ulc_qwn_v = Matrix([0, 0, 1])
ulc_ndc_v = Matrix([-1, 1, 1])
assert ndc_X_qwn_m * ulc_qwn_v == ulc_ndc_v

# Upper right corner
urc_qwn_v = Matrix([w_win, 0, 1])
urc_ndc_v = Matrix([1, 1, 1])
assert ndc_X_qwn_m * urc_qwn_v == urc_ndc_v

# Lower left corner
llc_qwn_v = Matrix([0, h_win, 1])
llc_ndc_v = Matrix([-1, -1, 1])
assert ndc_X_qwn_m * llc_qwn_v == llc_ndc_v

# Lower right corner
lrc_qwn_v = Matrix([w_win, h_win, 1])
lrc_ndc_v = Matrix([1, -1, 1])
assert ndc_X_qwn_m * lrc_qwn_v == lrc_ndc_v

Eq(ndc_X_qwn_s, ndc_X_qwn_m, evaluate=False)

                 ⎡  2             ⎤
                 ⎢─────    0    -1⎥
                 ⎢w_win           ⎥
                 ⎢                ⎥
^{ndc}X__{qwn} = ⎢        -2      ⎥
                 ⎢  0    ─────  1 ⎥
                 ⎢       h_win    ⎥
                 ⎢                ⎥
                 ⎣  0      0    1 ⎦

In [14]:
# nic_X_ndc: transform to vimage normalized image coordinates from OpenGL normalized device coordinates

w_omp, h_omp = symbols("w_omp, h_omp")  # (reoriented) image dimensions
zoom = symbols("zoom")  # windows per image (default 1)
cen_x_omp, cen_y_omp = symbols("^{omp}cen_x ^{omp}cen_y")

# TODO: aspect, center
aspect_omp = w_omp / h_omp
aspect_qwn = w_win / h_win

# Case A: If aspect_omp > aspect_qwn, image is fatter than window, so pad top/bottom at zoom == 1
# Case B: otherwise, pad left/right at zoom == 1

aspect_scale_ompA = w_omp
aspect_scale_ompB = h_omp
aspect_scale_qwnA = w_win
aspect_scale_qwnB = h_win

# Create a symbol for the A/B choice of aspect scaling
asc_omp, asc_qwn = symbols("asc_omp asc_qwn")

aspect_scale = Matrix([
    [w_win / asc_qwn, 0, 0],
    [0, h_win / asc_qwn, 0],
    [0, 0, 1],
])

zoom_scale = Matrix([
    [1 / zoom, 0, 0],
    [0, 1 / zoom, 0],
    [0, 0, 1],
])


nic_X_ndc_m = zoom_scale * aspect_scale

Eq(nic_X_ndc_s, nic_X_ndc_m, evaluate=False)

                 ⎡   w_win                     ⎤
                 ⎢────────────       0        0⎥
                 ⎢asc_qwn⋅zoom                 ⎥
                 ⎢                             ⎥
^{nic}X__{ndc} = ⎢                 h_win       ⎥
                 ⎢     0        ────────────  0⎥
                 ⎢              asc_qwn⋅zoom   ⎥
                 ⎢                             ⎥
                 ⎣     0             0        1⎦

In [15]:
# omp_X_nic: Transform from normalized image coordinates to oriented image pixels

center_shift = Matrix([
    [1, 0, -2 * (w_omp / 2 - cen_x_omp) / asc_omp],
    [0, 1, 2 * (h_omp / 2 - cen_y_omp) / asc_omp],
    [0, 0, 1],
])

omp_X_nic_m = Matrix([
    [asc_omp / 2, 0, w_omp / 2],
    [0, -asc_omp / 2, h_omp / 2],
    [0, 0, 1],
]) * center_shift

Eq(omp_X_nic_s, omp_X_nic_m, evaluate=False)

                 ⎡ascₒₘₚ                       ⎤
                 ⎢──────     0      ^{omp}cenₓ ⎥
                 ⎢  2                          ⎥
                 ⎢                             ⎥
^{omp}X__{nic} = ⎢        -ascₒₘₚ              ⎥
                 ⎢  0     ────────  ^{omp}cen_y⎥
                 ⎢           2                 ⎥
                 ⎢                             ⎥
                 ⎣  0        0           1     ⎦

In [16]:
omp_X_qwn_m = omp_X_nic_m * nic_X_ndc_m * ndc_X_qwn_m
Eq(omp_X_qwn_s, omp_X_qwn_m, evaluate=False)

                 ⎡   ascₒₘₚ                                 ascₒₘₚ⋅w_win  ⎤
                 ⎢────────────       0        ^{omp}cenₓ - ────────────── ⎥
                 ⎢asc_qwn⋅zoom                             2⋅asc_qwn⋅zoom ⎥
                 ⎢                                                        ⎥
^{omp}X__{qwn} = ⎢                 ascₒₘₚ                    ascₒₘₚ⋅h_win ⎥
                 ⎢     0        ────────────  ^{omp}cen_y - ──────────────⎥
                 ⎢              asc_qwn⋅zoom                2⋅asc_qwn⋅zoom⎥
                 ⎢                                                        ⎥
                 ⎣     0             0                     1              ⎦

In [17]:
nic_X_qwn_s = symbols("^{nic}X^{qwn}")
Eq(nic_X_qwn_s, nic_X_ndc_m * ndc_X_qwn_m, evaluate=False)

                 ⎡     2                        -w_win    ⎤
                 ⎢────────────       0        ────────────⎥
                 ⎢asc_qwn⋅zoom                asc_qwn⋅zoom⎥
                 ⎢                                        ⎥
^{nic}X__{qwn} = ⎢                  -2           h_win    ⎥
                 ⎢     0        ────────────  ────────────⎥
                 ⎢              asc_qwn⋅zoom  asc_qwn⋅zoom⎥
                 ⎢                                        ⎥
                 ⎣     0             0             1      ⎦

In [18]:
# Confidence checks

# Test case for zoom factor 1
test_m = omp_X_qwn_m.subs(zoom, 1)
# Default image center
test_m = test_m.subs(cen_x_omp, w_omp/2).subs(cen_y_omp, h_omp/2)
# Square window, square image
test_m = test_m.subs(h_omp, w_omp).subs(h_win, w_win).subs(asc_omp, w_omp).subs(asc_qwn, w_win)
assert test_m * Matrix([0, 0, 1]) == Matrix([0, 0, 1])  # Upper left corner
assert test_m * Matrix([w_win, w_win, 1]) == Matrix([w_omp, w_omp, 1])  # Lower right corner
assert test_m * Matrix([w_win / 2, w_win / 2, 1]) == Matrix([w_omp / 2, w_omp / 2, 1])  # Center

# Test case for zoom factor 2
test_m = omp_X_qwn_m.subs(zoom, 2)
# Default image center
test_m = test_m.subs(cen_x_omp, w_omp/2).subs(cen_y_omp, h_omp/2)
# Square window, square image
test_m = test_m.subs(h_omp, w_omp).subs(h_win, w_win).subs(asc_omp, w_omp).subs(asc_qwn, w_win)
assert test_m * Matrix([w_win / 2, w_win / 2, 1]) == Matrix([w_omp / 2, w_omp / 2, 1])  # Center unchanged
assert test_m * Matrix([0, 0, 1]) == Matrix([w_omp / 4, w_omp / 4, 1])  # Upper left is offset by 1/4

# Test case for center at upper left
test_m = omp_X_qwn_m.subs(zoom, 1)
# image center at upper left
test_m = test_m.subs(cen_x_omp, 0).subs(cen_y_omp, 0)
# Square window, square image
test_m = test_m.subs(h_omp, w_omp).subs(h_win, w_win).subs(asc_omp, w_omp).subs(asc_qwn, w_win)
assert test_m * Matrix([0, 0, 1]) == Matrix([-w_omp/2, -w_omp/2, 1]) # Upper left of window is in negative image coordinates
assert test_m * Matrix([w_win, w_win, 1]) == Matrix([w_omp/2, w_omp/2, 1]) # Lower right of window is in image center
assert test_m * Matrix([w_win/2, w_win/2, 1]) == Matrix([0, 0, 1])  # Window center is image center

# Default image center
test_m = omp_X_qwn_m.subs(cen_x_omp, w_omp/2).subs(cen_y_omp, h_omp/2)
assert test_m * Matrix([w_win/2, h_win/2, 1]) == Matrix([w_omp/2, h_omp/2, 1]) # Window center is image center

In [19]:
# Jacobian for dragging in rectangular images
d_omp_x, d_omp_y = symbols("{\partial}^{omp}p_x, {\partial}^{omp}p_y")
d_qwn_x, d_qwn_y = symbols("{\partial}^{qwn}p_x, {\partial}^{qwn}p_y")
ue = UnevaluatedExpr
Eq(Matrix([d_omp_x, d_omp_y]), ue(omp_X_qwn_m[0:2,0:2]) * ue(Matrix([d_qwn_x, d_qwn_y]).T), evaluate=False)

                         ⎡   ascₒₘₚ                 ⎤                         
                         ⎢────────────       0      ⎥                         
⎡{\partial}_x__{omp}p⎤   ⎢asc_qwn⋅zoom              ⎥                         
⎢                    ⎥ = ⎢                          ⎥⋅[{\partial}_x__{qwn}p  {
⎣{\partial}_y__{omp}p⎦   ⎢                 ascₒₘₚ   ⎥                         
                         ⎢     0        ────────────⎥                         
                         ⎣              asc_qwn⋅zoom⎦                         

                    
                    
                    
\partial}_y__{qwn}p]
                    
                    
                    

In [20]:
# Final expressions for the change in image pixels for a change in window pixels.
# Increment the image center by the negative of this change, to pan the image with the mouse.
Eq(d_omp_x, omp_X_qwn_m[0,0] * d_qwn_x, evaluate=False), Eq(d_omp_y, omp_X_qwn_m[1,1] * d_qwn_y, evaluate=False)

In [21]:
# For 360 images, we want to pause at the nic coordinates, 
#  then add projection-specific transforms for each 360 projection
nic_X_qwn_s = symbols("^{nic}X^{qwn}")
nic_X_qwn_m = nic_X_ndc_m * ndc_X_qwn_m
Eq(nic_X_qwn_s, nic_X_qwn_m, evaluate=False)

                 ⎡     2                        -w_win    ⎤
                 ⎢────────────       0        ────────────⎥
                 ⎢asc_qwn⋅zoom                asc_qwn⋅zoom⎥
                 ⎢                                        ⎥
^{nic}X__{qwn} = ⎢                  -2           h_win    ⎥
                 ⎢     0        ────────────  ────────────⎥
                 ⎢              asc_qwn⋅zoom  asc_qwn⋅zoom⎥
                 ⎢                                        ⎥
                 ⎣     0             0             1      ⎦

In [22]:
# Projected 360 prj coordinates differ from nic by an isotropic scale
prj_X_nic_s = symbols("^{prj}X^{nic}")
prj_X_nic_m = Matrix([
    [pi/2, 0, 0],
    [0, pi/2, 0],
    [0, 0, 1],
])
Eq(prj_X_nic_s, prj_X_nic_m, evaluate=False)

                 ⎡π      ⎤
                 ⎢─  0  0⎥
                 ⎢2      ⎥
                 ⎢       ⎥
^{prj}X__{nic} = ⎢   π   ⎥
                 ⎢0  ─  0⎥
                 ⎢   2   ⎥
                 ⎢       ⎥
                 ⎣0  0  1⎦

In [23]:
prj_X_qwn_s = symbols("^{prj}X^{nic}")
prj_X_qwn_m = prj_X_nic_m * nic_X_qwn_m
Eq(prj_X_qwn_s, prj_X_qwn_m, evaluate=False)

                 ⎡     π                        -π⋅w_win    ⎤
                 ⎢────────────       0        ──────────────⎥
                 ⎢asc_qwn⋅zoom                2⋅asc_qwn⋅zoom⎥
                 ⎢                                          ⎥
^{prj}X__{nic} = ⎢                  -π           π⋅h_win    ⎥
                 ⎢     0        ────────────  ──────────────⎥
                 ⎢              asc_qwn⋅zoom  2⋅asc_qwn⋅zoom⎥
                 ⎢                                          ⎥
                 ⎣     0             0              1       ⎦

In [24]:
# Partial Jacobian matrix for dragging 360 images
nic_J_qwn_s = symbols("^{nic}J^{qwn}")
nic_J_qwn_m = nic_X_qwn_m[0:2,0:2]  # Just the scale, because it's linear or whatever.
Eq(nic_J_qwn_s, nic_J_qwn_m, evaluate=False)

                 ⎡     2                    ⎤
                 ⎢────────────       0      ⎥
                 ⎢asc_qwn⋅zoom              ⎥
^{nic}J__{qwn} = ⎢                          ⎥
                 ⎢                  -2      ⎥
                 ⎢     0        ────────────⎥
                 ⎣              asc_qwn⋅zoom⎦

In [25]:
# Projected unzoomed coordinates for 360 are scaled to +- pi/2 on screen
prj_J_qwn_s = symbols("^{prj}J^{qwn}")
prj_J_qwn_m = nic_J_qwn_m * pi / 2
prj_J_qwn_m
Eq(prj_J_qwn_s, prj_J_qwn_m, evaluate=False)

                 ⎡     π                    ⎤
                 ⎢────────────       0      ⎥
                 ⎢asc_qwn⋅zoom              ⎥
^{prj}J__{qwn} = ⎢                          ⎥
                 ⎢                  -π      ⎥
                 ⎢     0        ────────────⎥
                 ⎣              asc_qwn⋅zoom⎦

In [26]:
# Equirectangular coordinates - heading and pitch (in radians)
# heading is radians clockwise from north/straight-ahead when viewed from above
# pitch is radians above the horizon

# obq is xyz unit sphere coordinates "oblique" because a view transform may have been applied already
# right-handed x-positive-right, y-positive-up, z-positive-nearer
x_obq, y_obq, z_obq = symbols("^{obq}p_x, ^{obq}p_y, ^{obq}p_z")
p_obq_s = symbols("^{obq}p")

eqr_from_obq = Matrix([
    atan2(x_obq, -z_obq),  # negative Z, because the central pixel is at the far end of the sphere
    asin(y_obq),
])

p_eqr_s = symbols("^{eqr}p")
Eq(p_eqr_s, eqr_from_obq, evaluate=False)

          ⎡atan2(^{obq}pₓ, -^{obq}p_z)⎤
^{eqr}p = ⎢                           ⎥
          ⎣      asin(^{obq}p_y)      ⎦

In [27]:
eqr_x, eqr_y = symbols("^{eqr}p_x, ^{eqr}p_y")
obq_from_eqr = Matrix([
    [cos(eqr_y) * sin(eqr_x)],
    [sin(eqr_y)],
    [-cos(eqr_y) * cos(eqr_x)],
])

Eq(p_obq_s, obq_from_eqr, evaluate=False)

          ⎡sin(^{eqr}pₓ)⋅cos(^{eqr}p_y) ⎤
          ⎢                             ⎥
^{obq}p = ⎢       sin(^{eqr}p_y)        ⎥
          ⎢                             ⎥
          ⎣-cos(^{eqr}pₓ)⋅cos(^{eqr}p_y)⎦

In [28]:
# Stereographic coordinates
# Assume that ste coordinates range from -1 to +1 at zoom==1

# 2D Stereographic image coordinates. Range is unlimited.
ste_x, ste_y = symbols("^{ste}p_x, ^{ste}p_y")

d = ste_x**2 + ste_y**2 + 4
obq_from_ste = Matrix([
    [4 * ste_x / d],
    [4 * ste_y / d],
    [(d - 8) / d],
])

Eq(p_obq_s, obq_from_ste, evaluate=False)

          ⎡        4⋅^{ste}pₓ        ⎤
          ⎢──────────────────────────⎥
          ⎢        2            2    ⎥
          ⎢^{ste}pₓ  + ^{ste}p_y  + 4⎥
          ⎢                          ⎥
          ⎢       4⋅^{ste}p_y        ⎥
          ⎢──────────────────────────⎥
^{obq}p = ⎢        2            2    ⎥
          ⎢^{ste}pₓ  + ^{ste}p_y  + 4⎥
          ⎢                          ⎥
          ⎢        2            2    ⎥
          ⎢^{ste}pₓ  + ^{ste}p_y  - 4⎥
          ⎢──────────────────────────⎥
          ⎢        2            2    ⎥
          ⎣^{ste}pₓ  + ^{ste}p_y  + 4⎦

In [29]:
# Stereographic Jacobian obq_J_ste
obq_J_ste_m = Matrix([
    [diff(obq_from_ste[0], ste_x), diff(obq_from_ste[0], ste_y)],
    [diff(obq_from_ste[1], ste_x), diff(obq_from_ste[1], ste_y)],
    [diff(obq_from_ste[2], ste_x), diff(obq_from_ste[2], ste_y)],
])
dp_obq_x, dp_obq_y, dp_obq_z = symbols("^{obq}{\partial}p_x ^{obq}{\partial}p_y ^{obq}{\partial}p_z")
dp_ste_x, dp_ste_y = symbols("^{ste}{\partial}p_x ^{ste}{\partial}p_y")
dp_obq = Matrix([dp_obq_x, dp_obq_y, dp_obq_z])
dp_ste = Matrix([dp_ste_x, dp_ste_y])
Eq(dp_obq, ue(obq_J_ste_m) * ue(dp_ste.transpose()), evaluate=False)

                        ⎡                          2                          
                        ⎢                8⋅^{ste}pₓ                         4 
                        ⎢     - ───────────────────────────── + ──────────────
                        ⎢                                   2           2     
                        ⎢       ⎛        2            2    ⎞    ^{ste}pₓ  + ^{
                        ⎢       ⎝^{ste}pₓ  + ^{ste}p_y  + 4⎠                  
                        ⎢                                                     
                        ⎢                                                     
⎡^{obq}{\partial}pₓ ⎤   ⎢                        -8⋅^{ste}pₓ⋅^{ste}p_y        
⎢                   ⎥   ⎢                    ─────────────────────────────    
⎢^{obq}{\partial}p_y⎥ = ⎢                                                2    
⎢                   ⎥   ⎢                    ⎛        2            2    ⎞     
⎣^{obq}{\partial}p_z⎦   ⎢                    ⎝^{ste}